In [1]:
import sys
import os

# Source: https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce

from lui.ecoc import SVMRandomInvariantsECOC
from lui.types import InvariantTypes
from utils import run_experiment

In [2]:
df = pd.read_csv("../data/ecoli.data", header=None)
df

,0,1,2,3,4,5,6,7,8
0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp
...,...,...,...,...,...,...,...,...,...
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30,pp
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32,pp
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38,pp
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37,pp


In [3]:
X, y = df.iloc[:, :-1], df.iloc[:, -1].values
X

,0,1,2,3,4,5,6,7
0,AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35
1,ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44
2,ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46
3,ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36
4,ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35
...,...,...,...,...,...,...,...,...
331,TREA_ECOLI,0.74,0.56,0.48,0.5,0.47,0.68,0.30
332,UGPB_ECOLI,0.71,0.57,0.48,0.5,0.48,0.35,0.32
333,USHA_ECOLI,0.61,0.60,0.48,0.5,0.44,0.39,0.38
334,XYLF_ECOLI,0.59,0.61,0.48,0.5,0.42,0.42,0.37


In [4]:
# Transform labels to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
hash_encoder = ce.HashingEncoder()
X = hash_encoder.fit_transform(X)
X = X.values
X

array([[0.  , 0.  , 0.  , ..., 0.56, 0.24, 0.35],
       [0.  , 0.  , 0.  , ..., 0.54, 0.35, 0.44],
       [1.  , 0.  , 0.  , ..., 0.49, 0.37, 0.46],
       ...,
       [1.  , 0.  , 0.  , ..., 0.44, 0.39, 0.38],
       [0.  , 0.  , 0.  , ..., 0.42, 0.42, 0.37],
       [0.  , 0.  , 0.  , ..., 0.31, 0.53, 0.52]])

In [6]:
num_dimensions = X.shape[1]
num_classes = len(np.unique(y))

print("Num dimensions: ", num_dimensions)
print("Num classes: ", num_classes)

Num dimensions:  15
Num classes:  8


In [7]:
encoding = np.eye(num_classes)
encoding

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [8]:
# Define models
models = [
    ('Baseline', SVMRandomInvariantsECOC(encoding)),
    ('Vapnik', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.VAPNIK, tolerance=20)),
    ('Projections', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.PROJECTION, tolerance=20)),
    ('Hyperplanes', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.HYPERPLANE, tolerance=20)),
]

In [9]:
# Define models parameters
models_parameters = [
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': [0],
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 2),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    }
]

In [10]:
# Define seeds and train sizes
seeds = [47, 4, 81]
train_sizes = [1.0, 0.5, 0.1]

In [12]:
results_df = run_experiment(X, y, seeds, train_sizes, models, models_parameters)
results_df.to_csv('results/ecoli.csv', index=False)

-------------------- EXPERIMENT 1 --------------------




------ Using 1.0 of the training data ------


Training Baseline model

Running experiment with random_state=47


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.01, num_invariants=0, random_state=47)
Accuracy:  0.8235294117647058
Running experiment with random_state=4


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.01, num_invariants=0, random_state=4)
Accuracy:  0.8235294117647058
Running experiment with random_state=81


...

Best estimator:  SVMRandomInvariantsECOC(C=0.01,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.01,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=9, random_state=47, tolerance=20)
Accuracy:  0.6617647058823529
Running experiment with random_state=4


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001, delta=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        gamma=0.01,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=1, random_state=4, tolerance=20)
Accuracy:  0.7058823529411765
Running experiment with random_state=81


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=1.0,
                        encoding=array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]]),
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=10, random_state=81, tolerance=20)
Accuracy:  0.6911764705882353



